#FOOD DELIVERY TIME PREDICTION USING ENSEMBLE LEARNING

##Basic things to be known about this model.
1. Here in our dataset column, we can see that the food delivery time will have commonly seven approximate timings and we are converting them into as seven categories. 
2. They are 10, 20, 45, 30, 65, 80 and 120 in minutes. 
  

##Purpose of this Model

1. Initially, we are using classifier models like Decision Tree, SVC and  Random forest  and predicting its accurancy. 
2. Then we are using ensembling techniques like Bagging, AdaBoost, to improve the model's accurance. 
3. Also we are combining all the model and using voting Classifier (hard voting) and getting the best classifier.

##Importing

###Importing the necessary libraries

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from time import time
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier, VotingClassifier
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.model_selection import train_test_split,GridSearchCV

###Importing the required Dataset

In [ ]:
dataset = pd.read_excel("/content/Data_Train.xlsx")

##Data Cleaning and Transformation

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Restaurant     11094 non-null  object
 1   Location       11094 non-null  object
 2   Cuisines       11094 non-null  object
 3   Average_Cost   11094 non-null  object
 4   Minimum_Order  11094 non-null  object
 5   Rating         11094 non-null  object
 6   Votes          11094 non-null  object
 7   Reviews        11094 non-null  object
 8   Delivery_Time  11094 non-null  object
dtypes: object(9)
memory usage: 780.2+ KB


###Label encoding and numeric transformation

In [ ]:
le = LabelEncoder()

In [ ]:
dataset['Restaurant']=le.fit_transform(dataset['Restaurant'])
dataset['Location']=le.fit_transform(dataset['Location'])

In [ ]:
dataset['Minimum_Order']=pd.to_numeric(dataset['Minimum_Order'].str.replace('₹',' '))

In [ ]:
dataset['Rating']=pd.to_numeric(dataset['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))

In [ ]:
dataset['Votes']=pd.to_numeric(dataset['Votes'].apply(lambda x : np.nan if x=='-' else x))
dataset['Reviews']=pd.to_numeric(dataset['Reviews'].apply(lambda x : np.nan if x=='-' else x))

In [ ]:
dataset['Cuisines']=le.fit_transform(dataset['Cuisines'])

In [ ]:
dataset['Average_Cost']=pd.to_numeric(dataset['Average_Cost'].str.replace('[^0-9]',''))
dataset['Delivery_Time']=pd.to_numeric(dataset['Delivery_Time'].str.replace('[^0-9]',''))

<ipython-input-214-c22157780a90>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['Average_Cost']=pd.to_numeric(dataset['Average_Cost'].str.replace('[^0-9]',''))
<ipython-input-214-c22157780a90>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['Delivery_Time']=pd.to_numeric(dataset['Delivery_Time'].str.replace('[^0-9]',''))


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11094 entries, 0 to 11093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Restaurant     11094 non-null  int64  
 1   Location       11094 non-null  int64  
 2   Cuisines       11094 non-null  object 
 3   Average_Cost   11093 non-null  float64
 4   Minimum_Order  11094 non-null  int64  
 5   Rating         9131 non-null   float64
 6   Votes          9020 non-null   float64
 7   Reviews        8782 non-null   float64
 8   Delivery_Time  11094 non-null  int64  
dtypes: float64(4), int64(4), object(1)
memory usage: 780.2+ KB


In [ ]:
dataset.head(10)

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,5086,10,"Fast Food, Rolls, Burger, Salad, Wraps",200.0,50,3.5,12.0,4.0,30
1,1785,30,"Ice Cream, Desserts",100.0,50,3.5,11.0,4.0,30
2,571,19,"Italian, Street Food, Fast Food",150.0,50,3.6,99.0,30.0,65
3,4708,28,"Mughlai, North Indian, Chinese",250.0,99,3.7,176.0,95.0,30
4,4896,26,"Cafe, Beverages",200.0,99,3.2,521.0,235.0,65
5,4030,26,"South Indian, North Indian, Chinese",150.0,50,3.8,46.0,18.0,30
6,2649,19,"Beverages, Fast Food",150.0,50,3.7,108.0,31.0,30
7,6168,8,"Chinese, Thai, Asian",650.0,50,4.0,1731.0,1235.0,45
8,1871,8,"Mithai, Street Food",100.0,50,3.9,110.0,26.0,30
9,2353,28,"Fast Food, North Indian, Rolls, Chinese, Momos...",200.0,50,3.9,562.0,294.0,65


###Finding the missing values

In [ ]:
# Count the number of missing values in each column
print(dataset.isnull().sum())

Restaurant          0
Location            0
Cuisines            0
Average_Cost        1
Minimum_Order       0
Rating           1963
Votes            2074
Reviews          2312
Delivery_Time       0
dtype: int64


###Filling the Missing Values

In [ ]:
a_mean = dataset["Average_Cost"].mean()

In [ ]:
dataset["Average_Cost"] = dataset["Average_Cost"].fillna(a_mean)

In [ ]:
dataset["Average_Cost"].isnull().sum()

0

In [ ]:
# Calculate the mean of the ratings column
r_median = dataset['Rating'].median() 

In [ ]:
# Use the .fillna() method to replace null values with the calculated mean value
dataset['Rating'] = dataset['Rating'].fillna(r_median)

In [ ]:
dataset['Rating'].isnull().sum()

0

In [ ]:
r_mode = dataset['Reviews'].mean()

In [ ]:
# Use the .fillna() method to replace null values with the calculated mode value
dataset['Reviews'] = dataset['Reviews'].fillna(r_mode)

In [ ]:
dataset['Reviews'].isnull().sum()

0

In [ ]:
v_median = dataset['Votes'].median()

In [ ]:
# Use the .fillna() method to replace null values with the calculated mode value
dataset['Votes'] = dataset['Votes'].fillna(v_median)

In [ ]:
dataset['Votes'].isnull().sum()

0

##Finding the Categories of the Target variable

In [ ]:
dataset['Delivery_Time'].unique()

array([ 30,  65,  45,  10,  20, 120,  80])

##Performing the pre-Requisite for modeling

Separating the dependent and independent variables 

In [ ]:
x=dataset.drop(['Delivery_Time', 'Restaurant', 'Location'],axis=1)
y=dataset['Delivery_Time'].astype('category')

In [ ]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 11094 entries, 0 to 11093
Series name: Delivery_Time
Non-Null Count  Dtype   
--------------  -----   
11094 non-null  category
dtypes: category(1)
memory usage: 11.3 KB


In [ ]:
x

,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,995,200.0,50,3.5,12.0,4.000000
1,1116,100.0,50,3.5,11.0,4.000000
2,1182,150.0,50,3.6,99.0,30.000000
3,1398,250.0,99,3.7,176.0,95.000000
4,388,200.0,99,3.2,521.0,235.000000
...,...,...,...,...,...,...
11089,2166,250.0,50,4.2,326.0,189.000000
11090,860,100.0,50,3.6,36.0,16.000000
11091,2036,100.0,50,3.5,45.0,18.000000
11092,1412,100.0,50,3.1,24.0,9.000000


Splitting the Dataset for evaluating the model

In [ ]:
dataset.reset_index(drop=True, inplace=True)

In [ ]:
pd.DataFrame(y)

,Delivery_Time
0,30
1,30
2,65
3,30
4,65
...,...
11089,30
11090,30
11091,30
11092,30


###Splitting the Dataset for training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

##Defining the various classifier Models

###Base Estimator
Creating the Base Models

In [ ]:
# Define base models
dtc = DecisionTreeClassifier(random_state=42)
svc = SVC(probability=True, random_state=42)
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

###Bagging Classifier:
Defining the bagging classifier with the above mentioned base models.

In [ ]:
# Define bagging classifiers for each base model
dtc_bc = BaggingClassifier(estimator=dtc, n_estimators=10, random_state=42)
svc_bc = BaggingClassifier(estimator=svc, n_estimators=10, random_state=42)
rfc_bc = BaggingClassifier(estimator=rfc, n_estimators=10, random_state=42)

###AdaBoost Classifier:
Defining the AdaBoost classifier with the above mentioned base models.

In [ ]:
# Define boosting classifiers for each base model
dtc_abc = AdaBoostClassifier(estimator=dtc, n_estimators=10, random_state=42)
svc_abc = AdaBoostClassifier(estimator=svc, n_estimators=10, random_state=42)
rfc_abc = AdaBoostClassifier(estimator=rfc, n_estimators=10, random_state=42)

###Voting Classifier:
Defining the voting classifier (Hard Voting) with the above mentioned classifier models.

In [ ]:
# Define voting classifier
vc = VotingClassifier(estimators=[
    ('dtc', dtc),
    ('dtc_bc', dtc_bc),
    ('dtc_abc', dtc_abc),
    ('svc', svc),
    ('svc_bc', svc_bc),
    ('svc_abc', svc_abc),
    ('rfc', rfc),
    ('rfc_bc', rfc_bc),
    ('rfc_abc', rfc_abc)
], voting='hard')

##Evaluating the defined Models

###1. Base models and its accuracy

In [ ]:
# Train and evaluate the base models
models = [dtc, svc, rfc]
for model in models:
    model.fit(X_train, y_train)
    print("Accuracy of {}: {}".format(type(model).__name__, model.score(X_test, y_test)))

Accuracy of DecisionTreeClassifier: 0.743578188373141
Accuracy of SVC: 0.7138350608382155
Accuracy of RandomForestClassifier: 0.8012618296529969


###2. Bagging Classifiers and its accuracy|

In [ ]:
# Train and evaluate the bagging classifiers
models_bc = [dtc_bc, svc_bc, rfc_bc]
for model_bc in models_bc:
    model_bc.fit(X_train, y_train)
    print("Accuracy of {} with Bagging: {}".format(type(model_bc.estimator).__name__, model_bc.score(X_test, y_test)))

Accuracy of DecisionTreeClassifier with Bagging: 0.7809824245155476
Accuracy of SVC with Bagging: 0.711581793600721
Accuracy of RandomForestClassifier with Bagging: 0.7890941865705272


###3. Boosting classifier and its accuracy

In [ ]:
# Train and evaluate the boosting classifiers
models_abc = [dtc_abc, svc_abc, rfc_abc]
for model_abc in models_abc:
    model_abc.fit(X_train, y_train)
    print("Accuracy of {} with Boosting: {}".format(type(model_abc.estimator).__name__, model_abc.score(X_test, y_test)))

Accuracy of DecisionTreeClassifier with Boosting: 0.7854889589905363
Accuracy of SVC with Boosting: 0.6804867057232988
Accuracy of RandomForestClassifier with Boosting: 0.8021631365479945


###Finding the Accuracy of ensembled model

In [ ]:
# Train and evaluate the voting classifier
vc.fit(X_train, y_train)
print("Accuracy of Ensemble VotingClassifier: {}".format(vc.score(X_test, y_test)))

Accuracy of Ensemble VotingClassifier: 0.7981072555205048


##INFERENCE

From the accuracy that we got from the above models, we can find that, **RandomForest Classifier with Boosting** have 80% Accuracy. So we can use this model for future classifications